In [6]:
import pandas as pd
import glob

In [2]:
# 步骤1: 读取CSV文件
df = pd.read_csv('author&funding_info.csv')

# 步骤2: 创建一个新列`expiration_year`
df['expiration_year'] = df['expiration_date'].str[-4:]

# 步骤3: 保留指定的列
columns_to_keep = [
    'email', 'institution', 'interests', 'award_year', 'expiration_year',
    'award_amount', 'award_title', 'total_citations', 'h_index',
    'citation_2011', 'citation_2012', 'citation_2013', 'citation_2014',
    'citation_2015', 'citation_2016', 'citation_2017', 'citation_2018',
    'citation_2019', 'citation_2020'
]

# 确保所有指定列都存在于DataFrame中，以避免KeyError
columns_to_keep = [col for col in columns_to_keep if col in df.columns]

# 重新定义DataFrame，只包含指定的列
df = df[columns_to_keep]
df.head()

,email,institution,interests,award_year,expiration_year,award_amount,award_title,total_citations,h_index,citation_2011,citation_2012,citation_2013,citation_2014,citation_2015,citation_2016,citation_2017,citation_2018,citation_2019,citation_2020
0,talalasad@earthlink.net,CUNY Graduate School University Center,['organic chemistry'],2011,2012,19901,Doctoral Dissertation Research: Mental Disorde...,106,3,2.0,6.0,4.0,6.0,12.0,13.0,10.0,11.0,13.0,5.0
1,vanboven@colorado.edu,University of Colorado at Boulder,"['Social psychology', 'judgment and decision m...",2011,2015,250000,EAGER: Perceiving Political Distributions,12916,46,363.0,523.0,600.0,621.0,745.0,759.0,782.0,921.0,868.0,1234.0
2,bedricks@ohsu.edu,Oregon Health & Science University,"['NLP', 'Medical Informatics']",2011,2013,114140,"Corpora of Non-Linguistic Symbol Systems, and ...",2230,21,63.0,54.0,76.0,74.0,84.0,70.0,87.0,105.0,153.0,253.0
3,ost@psu.edu,Pennsylvania State Univ University Park,"['Archaeology (eastern North America', 'Warfar...",2011,2015,113625,Late Prehistoric Warfare in Eastern North Amer...,9549,42,295.0,366.0,379.0,458.0,425.0,479.0,582.0,601.0,563.0,633.0
4,gable@psych.ucsb.edu,University of California-Santa Barbara,NaN,2011,2015,377951,Capitalizing on Positive Events,30300,54,1119.0,1414.0,1563.0,1701.0,2017.0,1811.0,2022.0,2133.0,2204.0,2186.0


In [3]:
# 扩展每个作者为10行，添加year列
expanded_rows = []
for _, row in df.iterrows():
    for year in range(2011, 2021):
        new_row = row.copy()
        new_row['year'] = year
        # 更新award_amount
        if year >= int(row['award_year']) and year <= int(row['expiration_year']):
            new_row['award_amount'] = row['award_amount']
        else:
            new_row['award_amount'] = 0
        expanded_rows.append(new_row)

expanded_df = pd.DataFrame(expanded_rows)

# 合并重复作者的奖励金额
grouped_df = expanded_df.groupby(['email', 'year'], as_index=False).agg({
    'institution': 'first',
    'interests': 'first',
    'award_year': 'first',
    'expiration_year': 'first',
    'award_amount': 'sum',
    'award_title': 'first',
    'total_citations': 'first',
    'h_index': 'first',
    **{f'citation_{year}': 'first' for year in range(2011, 2021)}
})

# 创建citation列
for year in range(2011, 2021):
    grouped_df.loc[grouped_df['year'] == year, 'citation'] = grouped_df[f'citation_{year}']

# 删除不再需要的citation_XXXX列
citation_columns = [f'citation_{year}' for year in range(2011, 2021)]
grouped_df.drop(columns=citation_columns, inplace=True)

df.head()

,email,institution,interests,award_year,expiration_year,award_amount,award_title,total_citations,h_index,citation_2011,citation_2012,citation_2013,citation_2014,citation_2015,citation_2016,citation_2017,citation_2018,citation_2019,citation_2020
0,talalasad@earthlink.net,CUNY Graduate School University Center,['organic chemistry'],2011,2012,19901,Doctoral Dissertation Research: Mental Disorde...,106,3,2.0,6.0,4.0,6.0,12.0,13.0,10.0,11.0,13.0,5.0
1,vanboven@colorado.edu,University of Colorado at Boulder,"['Social psychology', 'judgment and decision m...",2011,2015,250000,EAGER: Perceiving Political Distributions,12916,46,363.0,523.0,600.0,621.0,745.0,759.0,782.0,921.0,868.0,1234.0
2,bedricks@ohsu.edu,Oregon Health & Science University,"['NLP', 'Medical Informatics']",2011,2013,114140,"Corpora of Non-Linguistic Symbol Systems, and ...",2230,21,63.0,54.0,76.0,74.0,84.0,70.0,87.0,105.0,153.0,253.0
3,ost@psu.edu,Pennsylvania State Univ University Park,"['Archaeology (eastern North America', 'Warfar...",2011,2015,113625,Late Prehistoric Warfare in Eastern North Amer...,9549,42,295.0,366.0,379.0,458.0,425.0,479.0,582.0,601.0,563.0,633.0
4,gable@psych.ucsb.edu,University of California-Santa Barbara,NaN,2011,2015,377951,Capitalizing on Positive Events,30300,54,1119.0,1414.0,1563.0,1701.0,2017.0,1811.0,2022.0,2133.0,2204.0,2186.0


In [5]:
grouped_df.to_csv('author&funding_info_expanded.csv', index=False)

In [8]:
# 定义要搜索的文件夹和模式
folder_path = 'publist/*.csv'

# 使用glob找到所有csv文件
csv_files = glob.glob(folder_path)

# 初始化一个空的DataFrame用于存放合并后的数据
merged_df = pd.DataFrame()

# 遍历找到的文件
for file in csv_files:
    # 读取CSV文件
    df = pd.read_csv(file, encoding='utf-8-sig')
    # 检查并合并数据
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# 筛选出Year列在2011到2020之间的行
filtered_df = merged_df[(merged_df['Year'] >= 2011) & (merged_df['Year'] <= 2020)]

# 保存最终的DataFrame到CSV文件
filtered_df.to_csv('merged_publications_2011_2020.csv', index=False, encoding='utf-8-sig')


In [15]:
df = pd.read_csv('author&funding_info.csv',encoding='utf-8-sig')
# 步骤2: 从Paper URL列提取user参数
# 使用apply和lambda函数从URL中提取user参数
filtered_df['user'] = filtered_df['Paper URL'].apply(lambda x: x.split('&')[2].split('=')[1])

# 步骤3: 在author_info_2016中找到对应的user，并获取email信息
# 创建一个字典，用于映射user到email
user_to_email = df.set_index('url')['email'].to_dict()

# 步骤4: 将email信息添加到pub_info_2013_2019的新列
# 使用map函数和刚刚创建的字典来映射user到email
filtered_df['email'] = filtered_df['user'].map(lambda x: user_to_email.get(f"https://scholar.google.com/citations?hl=en&user={x}"))
pub_info_df = filtered_df.drop(columns=['user'])

# 可选：保存更新后的filtered_df到CSV
pub_info_df.to_csv('updated_filtered_df.csv', index=False, encoding='utf-8-sig')

In [16]:
# 删除Title列和email列都相同的重复行
unique_df = pub_info_df.drop_duplicates(subset=['Title', 'email'], keep='first')

# 可选：保存更新后的DataFrame到CSV
unique_df.to_csv('unique_filtered_df.csv', index=False, encoding='utf-8-sig')

In [17]:
# 初始化新列
grouped_df['pub_num'] = 0
grouped_df['top_cited'] = 0.0

# 遍历grouped_df中的每一行
for index, row in grouped_df.iterrows():
    # 在unique_df中找到匹配的行
    matches = unique_df[(unique_df['email'] == row['email']) & (unique_df['Year'] == row['year'])]
    
    # 更新pub_num列
    grouped_df.at[index, 'pub_num'] = len(matches)
    
    # 如果有匹配的行，计算top_cited
    if len(matches) > 0:
        # 对Cited by列进行降序排序并取前三个的均值
        top_cited_avg = matches['Cited by'].nlargest(3).mean() if len(matches) >= 3 else matches['Cited by'].mean()
        grouped_df.at[index, 'top_cited'] = top_cited_avg
    else:
        # 如果没有匹配的行，top_cited为0
        grouped_df.at[index, 'top_cited'] = 0

# 可选：保存更新后的grouped_df到CSV
grouped_df.to_csv('updated_grouped_df.csv', index=False, encoding='utf-8-sig')